In [ ]:
# 🚀 Análise Ambiental Automatizada (SEMA-MT)
# 
# 1. Clique em "Executar tudo"
# 2. Quando aparecer a caixa de upload, envie o arquivo .zip com os arquivos do shapefile da fazenda
# 3. O resto será feito automaticamente: interseções, mapas e resultados para download
# 
# ✅ Arquivos obrigatórios dentro do .zip: .shp, .shx, .dbf, .prj


In [ ]:
!pip install geopandas owslib shapely matplotlib -q

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import zipfile, os
from google.colab import files
from shapely.geometry import Polygon, MultiPolygon
from matplotlib.patches import Polygon as MplPolygon
from matplotlib.collections import PatchCollection
from owslib.wfs import WebFeatureService

In [ ]:
print("📂 Envie agora o arquivo .zip do shapefile da fazenda...")
uploaded = files.upload()
for filename in uploaded:
    if filename.endswith(".zip"):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall("FAZENDA")

def find_shapefile(folder_path):
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".shp"):
                return os.path.join(root, file)
    return None

fazenda_fp = find_shapefile("FAZENDA")
fazenda = gpd.read_file(fazenda_fp).to_crs("EPSG:32721")
print("✅ Shapefile carregado com sucesso.")

In [ ]:
def baixar_wfs(tipo, nome):
    base_url = "https://geo.sema.mt.gov.br/geoserver/ows"
    layer = {
        "uso": "sema:uso_consolidado_2022",
        "radam": "sema:vegetacao_radambrasil_mt",
        "uc": "sema:unidade_conservacao",
        "ti": "sema:terra_indigena_funai_2020",
        "mun": "sema:limite_municipal"
    }[tipo]
    url = f"{base_url}?service=WFS&version=2.0.0&request=GetFeature&typeName={layer}&outputFormat=application/json"
    gdf = gpd.read_file(url).to_crs("EPSG:32721")
    gdf.to_file(f"{nome}.shp")
    return gdf

print("⏳ Baixando bases da SEMA...")
uso = baixar_wfs("uso", "uso_consolidado")
radam = baixar_wfs("radam", "radambrasil")
uc = baixar_wfs("uc", "unidades_conservacao")
ti = baixar_wfs("ti", "terras_indigenas")
mun = baixar_wfs("mun", "limites_municipais")
print("✅ Bases baixadas com sucesso.")

In [ ]:
def cor_por_fitoecolog(nome):
    nome = nome.lower()
    if 'floresta' in nome:
        return '#336633'
    elif 'cerrado' in nome:
        return '#f3e1b4'
    else:
        return '#dddddd'

print("🧮 Processando interseção com RADAMBRASIL...")
radam_clip = radam[radam.intersects(fazenda.unary_union)]
intersec_radam = gpd.overlay(radam_clip, fazenda, how='intersection')
intersec_radam = intersec_radam[intersec_radam.is_valid & ~intersec_radam.is_empty]
intersec_radam['cor'] = intersec_radam['FITOECOLOG'].apply(cor_por_fitoecolog)
intersec_radam['area_ha'] = intersec_radam.geometry.area / 10000
intersec_radam['buffer'] = intersec_radam.buffer(50)

fig, ax = plt.subplots(figsize=(12, 12))
fazenda.boundary.plot(ax=ax, color='black', linewidth=2.5)
for _, row in intersec_radam.iterrows():
    color = row['cor']
    geom = row['buffer']
    if isinstance(geom, Polygon):
        patch = MplPolygon(list(geom.exterior.coords))
        ax.add_patch(patch)
        patch.set_facecolor(color)
    elif isinstance(geom, MultiPolygon):
        for poly in geom.geoms:
            patch = MplPolygon(list(poly.exterior.coords))
            ax.add_patch(patch)
            patch.set_facecolor(color)
plt.title("Vegetação RADAMBRASIL")
plt.axis('off')
plt.savefig("mapa_radam.jpeg", dpi=300, bbox_inches='tight')
plt.show()
print("✅ Mapa RADAMBRASIL gerado.")

In [ ]:
print("🧮 Processando interseção com Uso Consolidado...")
intersec_uso = gpd.overlay(uso, fazenda, how='intersection')
intersec_uso['area_ha'] = intersec_uso.geometry.area / 10000

fig, ax = plt.subplots(figsize=(12, 12))
fazenda.boundary.plot(ax=ax, color='black', linewidth=2.5)
intersec_uso.plot(ax=ax, color='orange', edgecolor='black')
plt.title("Uso Consolidado (SEMA)")
plt.axis('off')
plt.savefig("mapa_uso.jpeg", dpi=300, bbox_inches='tight')
plt.show()
print("✅ Mapa Uso Consolidado gerado.")

In [ ]:
print("🧮 Processando sobreposição com UCs, TIs e Municípios...")
intersec_uc = gpd.overlay(uc, fazenda, how='intersection')
intersec_ti = gpd.overlay(ti, fazenda, how='intersection')
intersec_mun = gpd.overlay(mun, fazenda, how='intersection')
print("🌱 UC:", intersec_uc.area.sum() / 10000, "ha")
print("🧬 TI:", intersec_ti.area.sum() / 10000, "ha")
print("📍 Município:", intersec_mun.area.sum() / 10000, "ha")

In [ ]:
files.download("mapa_radam.jpeg")
files.download("mapa_uso.jpeg")
print("✅ Mapas prontos para download.")